In [4]:
!optimum-cli export onnx --model khanhld/wav2vec2-base-vietnamese-160h w2v2_onnx/

/home/spring/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/home/spring/miniconda3/lib/python3.12/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:872: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz, self.num_heads, tgt_len, self.head_dim):
		-[x] values not close enough, max diff: 8.33272933959961e-05 (atol: 1e-05)
The ONNX 

In [2]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from optimum.onnxruntime import ORTModelForCTC, ORTModelForSpeechSeq2Seq
from jiwer import wer 
model = ORTModelForCTC.from_pretrained("w2v2_onnx/")
print(type(model))
processor = Wav2Vec2Processor.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")


c:\Users\ADMIN\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'optimum.onnxruntime.modeling_ort.ORTModelForCTC'>


In [11]:
import torch
import librosa
import numpy as np
from jiwer import wer

In [30]:
#test model
def load_audio(file_path):
    speech_array, sampling_rate = librosa.load(file_path, sr=16000)
    inputs = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True).input_values
    return inputs

def transcribe(file_path):
    input_values = load_audio(file_path)
    print(input_values.shape)
    logits = model(input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)
    pred_transcript = processor.batch_decode(pred_ids)[0]
    
    return pred_transcript

def score(preds, true):
    print(f"predicted:{preds}")
    print(f"ground truth:{true}")
    score = wer(true, preds)
    print(f"WER:{score}")
    return score



In [ ]:
preds = transcribe("wav/2.wav")
# print(f"predicted:{preds}")
with open('txt/2.txt', 'r') as file:
    true = file.read()
    # print(f"ground truth:{true}")
# print(f"WER:{wer(preds, true)}")




score(preds, true)


## Evaluate performance:

In [25]:
#evaluate processing time
import time
times = []
for i in range(10):
    start = time.time()
    preds = transcribe(f"wav/{i}.wav")
    print(f"processing time:{time.time()-start}")
    print(f"transcribe: {preds}")
    end = time.time()
    times.append(end-start)


torch.Size([1, 139200])
processing time:0.7745380401611328
transcribe: hôm they hôm phu có nghĩa là chưa có cái giấy hôm thú nếu mà vậy thì cái cuộc bở lãnh phải làm coi như là cái người mà được ba lãnh họ phả ở nước ngoài học kế họ không
torch.Size([1, 50080])
processing time:0.20462536811828613
transcribe: theo bà đây là một sự thật đã được thừa nhận rộng rãi
torch.Size([1, 102080])
processing time:0.4502291679382324
transcribe: tướng ca pu chia còn yêu cầu thủ tướng sin ga po phải điều chỉnh phát biểu không đúng sự thật chút nào này theo lợi ông
torch.Size([1, 180320])
processing time:0.7073523998260498
transcribe: nó làm người dân mặc dù nhìn thủ rất bình thường nhớg mà rất rất là rộng ràng à thừ  từ từ tính tác đường phố và trên mỗi con mỗi con ngườn
torch.Size([1, 98240])
processing time:0.36347389221191406
transcribe: bây giờ mình nói từ khe xanh hai năm tới thể xanh vĩnh viễn ở mình đang ờ nói vì thẻ xanh nhưng mà có quá nhiều câu hỏi
torch.Size([1, 151360])
processing time:0.5

## Optimize the model 

In [5]:
from optimum.onnxruntime import ORTOptimizer, ORTModelForCTC, AutoOptimizationConfig
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")
model = ORTModelForCTC.from_pretrained("w2v2_onnx/")

optimizer = ORTOptimizer.from_pretrained(model)
save_dir_1 = 'w2v2_optim1'
save_dir_2 = 'w2v2_optim2'
save_dir_3 = 'w2v2_optim3'

optimization_config_1 = AutoOptimizationConfig.O1()
optimizer.optimize(save_dir=save_dir_1, optimization_config=optimization_config_1)

optimization_config_2 = AutoOptimizationConfig.O2()
optimizer.optimize(save_dir=save_dir_2, optimization_config=optimization_config_2)

optimization_config_3 = AutoOptimizationConfig.O3()
optimizer.optimize(save_dir=save_dir_3, optimization_config=optimization_config_3)




NotImplementedError: Tried to use ORTOptimizer for the model type wav2vec2, but it is not available yet. Please open an issue or submit a PR at https://github.com/huggingface/optimum.

## Quantize model

In [4]:
from optimum.onnxruntime import ORTQuantizer, ORTModelForCTC, AutoQuantizationConfig
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")
model = ORTModelForCTC.from_pretrained("w2v2_onnx/")
quantizer = ORTQuantizer.from_pretrained(model)

save_dir = 'w2v2_quant'
quantization_config = AutoQuantizationConfig.avx2(is_static=False, per_channel=False)
model_quantized_path = quantizer.quantize(save_dir=save_dir, quantization_config=quantization_config)


c:\Users\ADMIN\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#test quantized model
print(model_quantized_path)
model = ORTModelForCTC.from_pretrained("w2v2_quant/")
print(type(model))
processor = Wav2Vec2Processor.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")

preds = transcribe("vivos/test/waves/VIVOSDEV01/VIVOSDEV01_R002.wav")
print(preds)


w2v2_quant
<class 'optimum.onnxruntime.modeling_ort.ORTModelForCTC'>
torch.Size([1, 65000])
tiến cộc cạch chửn lợi có vổ những cớp sách


In [ ]:
folders = []
transcripts = []
filenames = []
with open('vivos/train/prompts.txt', 'r') as f:
    for line in f:
        filename, transcript = line.split(' ', 1)
        folder = filename.split('_')[0]
        print(f"folder: {folder}, transcript: {transcript}, filename: {filename}")
        folders.append(folder)
        transcripts.append(transcript)
        filenames.append(filename)




folder: VIVOSSPK01, transcript: KHÁCH SẠN
, filename: VIVOSSPK01_R001


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [32]:
print(transcripts)

['KHÁCH SẠN\n', 'CHỈ BẰNG CÁCH LUÔN NỖ LỰC THÌ CUỐI CÙNG BẠN MỚI ĐƯỢC ĐỀN ĐÁP\n', 'TRONG SỐ CÁC QUỐC GIA CÔNG NGHIỆP PHÁT TRIỂN\n', 'ANH ĐÃ NHÌN THẤY TRONG NHỮNG LẢI NHẢI DÔNG DÀI CỦA NHU\n', 'KHỦNG HOẢNG MÔI TRƯỜNG CẦN ĐƯỢC NGĂN CHẶN\n', 'TRÔNG THẤY TÀU CÁ ĐẰNG XA ANH ĐÃ ẤN CÒI CẢNH BÁO\n', 'MẤY SÀO RUỘNG CỦA GIA ĐÌNH ÔNG ĐÀNH BỎ HOANG CHO CỎ MỌC\n', 'CÓ ĐÊM GẶP PHẢI HEO NỌC NÓ RƯỢT\n', 'CÁC BÀN HỌC NHỎ HẰNG NGÀY NAY TRỞ THÀNH BÀN THỜ CỦA CÁC CHÁU\n', 'CHA MẸ TÔI THƯỜNG CHÌ CHIẾT NHAU BẰNG NHỮNG LỜI LẼ KHÓ NGHE\n', 'HẤP DẪN NHẤT LÀ CUỘC THI BƠI THUYỀN TRUYỀN THỐNG\n', 'VỚI NHỮNG CỜ LÍP NHẠC TRUYỀN HÌNH\n', 'MỘT TRONG NHỮNG CÁCH TỐT NHẤT\n', 'ĂN CẮP PHỖNG TƯỢNG\n', 'TỚI VỖ VAI HỎI VỀ CHIẾC ÁO MỚI MUA\n', 'BÀO CHẾ NHỮNG LOẠI THUỐC CHỮA NGHIỆN MA TÚY\n', 'NHỮNG MÁI NGÓI ÂM DƯƠNG CỔ KÍNH\n', 'MẶT MŨI NÓ CÒN CÀU CẠU\n', 'NẮNG ĐÀO LÀM ỬNG HỒNG NHỮNG ĐÔI MÁ THIẾU NỮ HÀ THÀNH\n', 'CHIẾC Ổ THƯỜNG XUYÊN ĐƯỢC KHÓA CHẶT VÀO BÀN\n', 'LÀM TÓC KIỂU NÀY TRÔNG BẠN QUÝ PHÁI MỘT CÁCH TỰ NHIÊN\n', 'ĐỒNG 

## Evaluation

In [ ]:
#evaluate quantized model
scores = []
for folder in folders:
    for filename in filenames:
        preds = transcribe(f"vivos/train/waves/{folder}/{filename}.wav")
        i = 0
        true = transcripts[i]
        print(type(true))
        evals  = score(preds, true)
        scores.append(evals)
        print(f"filename:{filename}, WER:{score}")
print(f"average WER:{np.mean(scores)}")

torch.Size([1, 48000])
<class 'str'>
predicted:hát sạn
ground truth:KHÁCH SẠN



TypeError: 'NoneType' object is not callable

## Give TVM a shot cuz why not? (NVM)

In [21]:
import onnx
import numpy as np
import tvm.relay as relay
import tvm
from tvm.contrib import graph_executor
import librosa
#load w2v2_onnx model
model_path = "w2v2_onnx/model.onnx"
onnx_model = onnx.load(model_path)



In [23]:
target = "llvm"
input_name = 'input'
input_data = load_audio("wav/2.wav")
shape_dict = {input_name: [0,0]}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

/home/spring/tvm/python/tvm/relay/frontend/onnx.py:7234: UserWarning: 'ORTModelForCTC' object has no attribute 'SerializeToString'
  warnings.warn(str(e))


AttributeError: 'ORTModelForCTC' object has no attribute 'graph'

In [16]:
import onnx

# Load your ONNX model
onnx_model = onnx.load(model_path)

# Print the input names
for input in onnx_model.graph.input:
    print(f"Input name: {input.name}")
    print(f"Input shape: {[dim.dim_value for dim in input.type.tensor_type.shape.dim]}")

Input name: input_values
Input shape: [0, 0]
